In [1]:
pip install googlemaps


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install openpyxl


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import time
import googlemaps
import pandas as pd
import csv
from pprint import pprint
import openpyxl
import json
import ast

In [2]:
API_KEY = open('API.txt','r').read()
map_client = googlemaps.Client(API_KEY)

Getting the cordinates of the all neighborhoods in the property dataset

In [ ]:
# List of neighborhood names in Baltimore
neighborhood_names = [
    "Mount Washington",
    "Fells Point",
    "Clifton",
    "Downtown",
    "Arlington",
    "Carroll",
    "Purnell",
    "Greater Northwood Covenant Association",
    "Northwood",
    "Locust Point",
    "Lakeland",
    "Bolton Hill",
    "Highlandtown",
    "Gwynn Oak",
    "Druid Hill Park",
    "Otterbein",
    "Franklintown",
    "Waverly",
    "Charles Village",
    "Old Goucher",
    "Cherry Hill",
    "South Baltimore",
    "Woodberry",
    "Govans",
    "Sharp Leadenhall",
    "Tuscany - Canterbury",
    "East Case",
    "Grove Park",
    "Mid-Town Belvedere",
    "New North Roland Park - Poplar Hill",
    "Mosher",
    "Middle East",
    "Riverside",
    "Garwyn Oaks",
    "Mount Holly",
    "Elwood Park",
    "Federal Hill-Montgomery",
    "West Forest Park",
    "Inner Harbor",
    "Ridgely's Delight",
    "Ednor Gardens - Lakeside",
    "Brooklyn",
    "Raspeburg",
    "Medfield",
    "Panway - Braddish",
    "Forest Park",
    "Harford - Echodale - Perring Parkway",
    "Station North",
    "Upper Fells Point",
    "Hampden",
    "Pen Lucy",
    "Oakenshawe",
    "Harwood",
    "Coldstream - Homestead - Montebello",
    "Bridgeview-Greenlawn",
    "Gay Street",
    "Franklin",
    "Pigtown",
    "Orangeville",
    "Better Waverly",
    "Butchers Hill",
    "Evergreen Lawn",
    "Morrell Park",
    "Glenham-Belford",
    "Western",
    "Madison - Eastend",
    "Walbrook",
    "Abell",
    "East Baltimore Midway",
    "Remington",
    "Coppin Heights",
    "Hollins Market",
    "Lauraville",
    "Mondawmin",
    "Canton",
    "Barre Circle",
    "Barclay",
    "Windsor Hills",
    "Wyndhurst",
    "Wyman Park",
    "Rosemont",
    "NW Community Action",
    "Winchester",
    "Woodring",
    "Curtis Bay",
    "Milton - Montford",
]

# Create an empty list to store neighborhood data
neighborhood_data = []

# Function to geocode neighborhood names and get coordinates and radii
def geocode_neighborhoods(names):
    for name in names:
        try:
            # Geocode the neighborhood name
            geocode_result = map_client.geocode(name + ', Baltimore, MD, USA')

            # Extracting latitude and longitude from the geocoding result
            if geocode_result and 'geometry' in geocode_result[0]:
                location = geocode_result[0]['geometry']['location']
                latitude = location['lat']
                longitude = location['lng']

                radius = 2000

                # Create a dictionary with neighborhood data
                neighborhood_info = {
                    "name": name,
                    "location": f"{latitude},{longitude}",
                    "radius": radius
                }

                neighborhood_data.append(neighborhood_info)
            else:
                print(f"Could not geocode {name}")

        except Exception as e:
            print(f"Error geocoding {name}: {str(e)}")

# Calling the function to geocode neighborhood names
geocode_neighborhoods(neighborhood_names)

# Creating a DataFrame from the neighborhood data
neighborhood_df = pd.DataFrame(neighborhood_data)

# Print the DataFrame or save it to a CSV file
print(neighborhood_df)
neighborhood_df.to_csv('neighborhood_data.csv', index=False)

Pulling allowed gyms data for Baltimore

In [ ]:
# Load the neighborhood details from a CSV file into a DataFrame
neighborhood_df = pd.read_csv('neighborhood_data.csv')

# Function to fetch gyms in a neighborhood
def fetch_gyms(neighborhood):
    allowed_columns = ["place_id", "geometry"]  # Specifying the allowed columns

    gym_list = []
    response = map_client.places_nearby(
        location=neighborhood["location"],
        radius=neighborhood["radius"],
        type='gym',  
    )

    while 'results' in response:
        for result in response['results']:
            # Filter columns to keep only allowed ones
            filtered_result = {key: result[key] for key in allowed_columns}

            # Add neighborhood information
            filtered_result['neighborhood'] = neighborhood["location"]

            gym_list.append(filtered_result)

        time.sleep(2)  # Pause to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=neighborhood["location"],
                radius=neighborhood["radius"],
                type='gym',
                page_token=response['next_page_token']
            )
        else:
            break

    return gym_list

# Fetch gyms for each neighborhood in the DataFrame
all_gyms = []

for index, row in neighborhood_df.iterrows():
    neighborhood_info = {
        "name": row["name"],
        "location": row["location"],
        "radius": row["radius"]
    }
    neighborhood_gyms = fetch_gyms(neighborhood_info)
    all_gyms.extend(neighborhood_gyms)

# Create a DataFrame from the combined list of gyms
gym_df = pd.DataFrame(all_gyms)

# Save the merged dataset to an Excel file
gym_df.to_excel('Allowed_gyms.xlsx', index=False)

Pulling allowed restaurant data for Baltimore

In [7]:
# Load the neighborhood details from a CSV file into a DataFrame
neighborhood_df = pd.read_csv('neighborhood_data.csv')

# Function to fetch restaurants in a neighborhood
def fetch_business(neighborhood):
    allowed_columns = ["place_id", "geometry"]  # Specifying the allowed columns

    restaurant_list = []
    response = map_client.places_nearby(
        location=neighborhood["location"],
        radius=neighborhood["radius"],
        type='restaurant',  
    )

    while 'results' in response:
        for result in response['results']:
            # Filter columns to keep only allowed ones
            filtered_result = {key: result[key] for key in allowed_columns}

            # Add neighborhood information
            filtered_result['neighborhood'] = neighborhood["location"]

            restaurant_list.append(filtered_result)

        time.sleep(2)  # Pause to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=neighborhood["location"],
                radius=neighborhood["radius"],
                type='restaurant',
                page_token=response['next_page_token']
            )
        else:
            break

    return restaurant_list

# Fetch restaurants for each neighborhood in the DataFrame
all_restaurants = []

for index, row in neighborhood_df.iterrows():
    neighborhood_info = {
        "name": row["name"],
        "location": row["location"],
        "radius": row["radius"]
    }
    neighborhood_restaurants = fetch_business(neighborhood_info)
    all_restaurants.extend(neighborhood_restaurants)

# Create a DataFrame from the combined list of restaurants
restaurant_df = pd.DataFrame(all_restaurants)

# Save the merged dataset to an Excel file
restaurant_df.to_excel('Allowed_restaurant.xlsx', index=False)

Pulling allowed banks data for Baltimore

In [8]:
# Load the neighborhood details from a CSV file into a DataFrame
neighborhood_df = pd.read_csv('neighborhood_data.csv')

# Function to fetch banks in a neighborhood
def fetch_banks(neighborhood):
    allowed_columns = ["place_id", "geometry"]  # Specifying the allowed columns

    bank_list = []
    response = map_client.places_nearby(
        location=neighborhood["location"],
        radius=neighborhood["radius"],
        type='bank',  
    )

    while 'results' in response:
        for result in response['results']:
            # Filter columns to keep only allowed ones
            filtered_result = {key: result[key] for key in allowed_columns}

            # Add neighborhood information
            filtered_result['neighborhood'] = neighborhood["location"]

            bank_list.append(filtered_result)

        time.sleep(2)  # Pause to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=neighborhood["location"],
                radius=neighborhood["radius"],
                type='bank',
                page_token=response['next_page_token']
            )
        else:
            break

    return bank_list

# Fetch banks for each neighborhood in the DataFrame
all_banks = []

for index, row in neighborhood_df.iterrows():
    neighborhood_info = {
        "name": row["name"],
        "location": row["location"],
        "radius": row["radius"]
    }
    neighborhood_banks = fetch_banks(neighborhood_info)
    all_banks.extend(neighborhood_banks)

# Create a DataFrame from the combined list of banks
bank_df = pd.DataFrame(all_banks)

# Save the merged dataset to an Excel file
bank_df.to_excel('Allowed_banks.xlsx', index=False)

I am noticing many duplicates in the place_id of the businessess, which is not a big deal in this scenario, but i am considering using a smaller search radius (1,500) around the neighbourhoods to see if the number of duplicates will reduce

Pulling allowed pharmacies data for Baltimore

In [4]:
# Load the neighborhood details from a CSV file into a DataFrame
neighborhood_df = pd.read_csv('neighborhood_data.csv')

# Function to fetch pharmacy in a neighborhood 
def fetch_pharmacy(neighborhood):
    allowed_columns = ["place_id", "geometry"]  # Specifying the allowed columns

    pharmacy_list = []
    response = map_client.places_nearby(
        location=neighborhood["location"],
        radius='1500',
        type='pharmacy',  
    )

    while 'results' in response:
        for result in response['results']:
            # Filter columns to keep only allowed ones
            filtered_result = {key: result[key] for key in allowed_columns}

            # Add neighborhood information
            filtered_result['neighborhood'] = neighborhood["location"]

            pharmacy_list.append(filtered_result)

        time.sleep(2)  # Pause to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=neighborhood["location"],
                radius='1500',
                type='pharmacy',
                page_token=response['next_page_token']
            )
        else:
            break

    return pharmacy_list

# Fetch pharmacy for each neighborhood in the DataFrame
all_pharmacy = []

for index, row in neighborhood_df.iterrows():
    neighborhood_info = {
        "name": row["name"],
        "location": row["location"],
        "radius": row["radius"]
    }
    neighborhood_pharmacy = fetch_pharmacy(neighborhood_info)
    all_pharmacy.extend(neighborhood_pharmacy)

# Create a DataFrame from the combined list of pharmacy
pharmacy_df = pd.DataFrame(all_pharmacy)

# Save the merged dataset to an Excel file
pharmacy_df.to_excel('Allowed_pharmacy.xlsx', index=False)

Pulling allowed supermarkets data for Baltimore

In [5]:
# Load the neighborhood details from a CSV file into a DataFrame
neighborhood_df = pd.read_csv('neighborhood_data.csv')

# Function to fetch supermarkets in a neighborhood 
def fetch_supermarket(neighborhood):
    allowed_columns = ["place_id", "geometry"]  # Specifying the allowed columns

    supermarket_list = []
    response = map_client.places_nearby(
        location=neighborhood["location"],
        radius='1500',
        type='supermarket',  
    )

    while 'results' in response:
        for result in response['results']:
            # Filter columns to keep only allowed ones
            filtered_result = {key: result[key] for key in allowed_columns}

            # Add neighborhood information
            filtered_result['neighborhood'] = neighborhood["location"]

            supermarket_list.append(filtered_result)

        time.sleep(2)  # Pause to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=neighborhood["location"],
                radius='1500',
                type='supermarket',
                page_token=response['next_page_token']
            )
        else:
            break

    return supermarket_list

# Fetch supermarkets for each neighborhood in the DataFrame
all_supermarket = []

for index, row in neighborhood_df.iterrows():
    neighborhood_info = {
        "name": row["name"],
        "location": row["location"],
        "radius": row["radius"]
    }
    neighborhood_supermarket = fetch_supermarket(neighborhood_info)
    all_supermarket.extend(neighborhood_supermarket)

# Create a DataFrame from the combined list of supermarkets
supermarket_df = pd.DataFrame(all_supermarket)

# Save the merged dataset to an Excel file
supermarket_df.to_excel('Allowed_supermarkets.xlsx', index=False)

Pulling allowed park data for Baltimore

In [6]:
# Load the neighborhood details from a CSV file into a DataFrame 
neighborhood_df = pd.read_csv('neighborhood_data.csv')

# Function to fetch parks in a neighborhood 
def fetch_park(neighborhood):
    allowed_columns = ["place_id", "geometry"]  # Specifying the allowed columns

    park_list = []
    response = map_client.places_nearby(
        location=neighborhood["location"],
        radius='1500',
        type='park',  
    )

    while 'results' in response:
        for result in response['results']:
            # Filter columns to keep only allowed ones
            filtered_result = {key: result[key] for key in allowed_columns}

            # Add neighborhood information
            filtered_result['neighborhood'] = neighborhood["location"]

            park_list.append(filtered_result)

        time.sleep(2)  # Pause to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=neighborhood["location"],
                radius='1500',
                type='park',
                page_token=response['next_page_token']
            )
        else:
            break

    return park_list

# Fetch parks for each neighborhood in the DataFrame
all_park = []

for index, row in neighborhood_df.iterrows():
    neighborhood_info = {
        "name": row["name"],
        "location": row["location"],
        "radius": row["radius"]
    }
    neighborhood_park = fetch_park(neighborhood_info)
    all_park.extend(neighborhood_park)

# Create a DataFrame from the combined list of parks
park_df = pd.DataFrame(all_park)

# Save the merged dataset to an Excel file
park_df.to_excel('Allowed_parks.xlsx', index=False)

 Pulling allowed night_club data for Baltimore

In [7]:
# Load the neighborhood details from a CSV file into a DataFrame 
neighborhood_df = pd.read_csv('neighborhood_data.csv')

# Function to fetch night_clubs in a neighborhood 
def fetch_night_club(neighborhood):
    allowed_columns = ["place_id", "geometry"]  # Specifying the allowed columns

    night_club_list = []
    response = map_client.places_nearby(
        location=neighborhood["location"],
        radius='1500',
        type='night_club',  
    )

    while 'results' in response:
        for result in response['results']:
            # Filter columns to keep only allowed ones
            filtered_result = {key: result[key] for key in allowed_columns}

            # Add neighborhood information
            filtered_result['neighborhood'] = neighborhood["location"]

            night_club_list.append(filtered_result)

        time.sleep(2)  # Pause to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=neighborhood["location"],
                radius='1500',
                type='night_club',
                page_token=response['next_page_token']
            )
        else:
            break

    return night_club_list

# Fetch night_clubs for each neighborhood in the DataFrame
all_night_club = []

for index, row in neighborhood_df.iterrows():
    neighborhood_info = {
        "name": row["name"],
        "location": row["location"],
        "radius": row["radius"]
    }
    neighborhood_night_club = fetch_night_club(neighborhood_info)
    all_night_club.extend(neighborhood_night_club)

# Create a DataFrame from the combined list of night_clubs
night_club_df = pd.DataFrame(all_night_club)

# Save the merged dataset to an Excel file
night_club_df.to_excel('Allowed_night_clubs.xlsx', index=False)

  Pulling allowed hospital data for Baltimore

In [8]:
# Load the neighborhood details from a CSV file into a DataFrame 
neighborhood_df = pd.read_csv('neighborhood_data.csv')

# Function to fetch hospitals in a neighborhood 
def fetch_hospital(neighborhood):
    allowed_columns = ["place_id", "geometry"]  # Specifying the allowed columns

    hospital_list = []
    response = map_client.places_nearby(
        location=neighborhood["location"],
        radius='1500',
        type='hospital',  
    )

    while 'results' in response:
        for result in response['results']:
            # Filter columns to keep only allowed ones
            filtered_result = {key: result[key] for key in allowed_columns}

            # Add neighborhood information
            filtered_result['neighborhood'] = neighborhood["location"]

            hospital_list.append(filtered_result)

        time.sleep(2)  # Pause to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=neighborhood["location"],
                radius='1500',
                type='hospital',
                page_token=response['next_page_token']
            )
        else:
            break

    return hospital_list

# Fetch hospitals for each neighborhood in the DataFrame
all_hospital = []

for index, row in neighborhood_df.iterrows():
    neighborhood_info = {
        "name": row["name"],
        "location": row["location"],
        "radius": row["radius"]
    }
    neighborhood_hospital = fetch_hospital(neighborhood_info)
    all_hospital.extend(neighborhood_hospital)

# Create a DataFrame from the combined list of hospitals
hospital_df = pd.DataFrame(all_hospital)

# Save the merged dataset to an Excel file
hospital_df.to_excel('Allowed_hospitals.xlsx', index=False)

  Pulling allowed bars data for Baltimore

In [3]:
# Load the neighborhood details from a CSV file into a DataFrame 
neighborhood_df = pd.read_csv('neighborhood_data.csv')

# Function to fetch bars in a neighborhood 
def fetch_bar(neighborhood):
    allowed_columns = ["place_id", "geometry"]  # Specifying the allowed columns

    bar_list = []
    response = map_client.places_nearby(
        location=neighborhood["location"],
        radius='1500',
        type='bar',  
    )

    while 'results' in response:
        for result in response['results']:
            # Filter columns to keep only allowed ones
            filtered_result = {key: result[key] for key in allowed_columns}

            # Add neighborhood information
            filtered_result['neighborhood'] = neighborhood["location"]

            bar_list.append(filtered_result)

        time.sleep(2)  # Pause to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=neighborhood["location"],
                radius='1500',
                type='bar',
                page_token=response['next_page_token']
            )
        else:
            break

    return bar_list

# Fetch bar for each neighborhood in the DataFrame
all_bar = []

for index, row in neighborhood_df.iterrows():
    neighborhood_info = {
        "name": row["name"],
        "location": row["location"],
        "radius": row["radius"]
    }
    neighborhood_bar = fetch_bar(neighborhood_info)
    all_bar.extend(neighborhood_bar)

# Create a DataFrame from the combined list of bars
bar_df = pd.DataFrame(all_bar)

# Save the merged dataset to an Excel file
bar_df.to_excel('Allowed_bars.xlsx', index=False)

  Pulling allowed beauty_salon data for Baltimore

In [4]:
# Load the neighborhood details from a CSV file into a DataFrame 
neighborhood_df = pd.read_csv('neighborhood_data.csv')

# Function to fetch beauty_salons in a neighborhood 
def fetch_beauty_salon(neighborhood):
    allowed_columns = ["place_id", "geometry"]  # Specifying the allowed columns

    beauty_salon_list = []
    response = map_client.places_nearby(
        location=neighborhood["location"],
        radius='1500',
        type='beauty_salon',  
    )

    while 'results' in response:
        for result in response['results']:
            # Filter columns to keep only allowed ones
            filtered_result = {key: result[key] for key in allowed_columns}

            # Add neighborhood information
            filtered_result['neighborhood'] = neighborhood["location"]

            beauty_salon_list.append(filtered_result)

        time.sleep(2)  # Pause to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=neighborhood["location"],
                radius='1500',
                type='beauty_salon',
                page_token=response['next_page_token']
            )
        else:
            break

    return beauty_salon_list

# Fetch beauty_salon for each neighborhood in the DataFrame
all_beauty_salon = []

for index, row in neighborhood_df.iterrows():
    neighborhood_info = {
        "name": row["name"],
        "location": row["location"],
        "radius": row["radius"]
    }
    neighborhood_beauty_salon = fetch_beauty_salon(neighborhood_info)
    all_beauty_salon.extend(neighborhood_beauty_salon)

# Create a DataFrame from the combined list of beauty_salons
beauty_salon_df = pd.DataFrame(all_beauty_salon)

# Save the merged dataset to an Excel file
beauty_salon_df.to_excel('Allowed_beauty_salons.xlsx', index=False)

Testing code to create a function for this code for next business types

In [3]:
# Load the neighborhood details from a CSV file into a DataFrame 
neighborhood_df = pd.read_csv('neighborhood_data.csv')

# General function to fetch businesses in a neighborhood 
def fetch_business(neighborhood, business_type):
    allowed_columns = ["place_id", "geometry"]  # Specifying the allowed columns

    business_list = []
    response = map_client.places_nearby(
        location=neighborhood["location"],
        radius='1500',
        type=business_type,  
    )

    while 'results' in response:
        for result in response['results']:
            # Filter columns to keep only allowed ones
            filtered_result = {key: result[key] for key in allowed_columns}

            # Add neighborhood information
            filtered_result['neighborhood'] = neighborhood["location"]

            business_list.append(filtered_result)

        time.sleep(2)  # Pause to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=neighborhood["location"],
                radius='1500',
                type=business_type,
                page_token=response['next_page_token']
            )
        else:
            break

    return business_list

def fetch_and_save_business_data(business_type, excel_filename):
    # Fetch businesses for each neighborhood in the DataFrame
    all_businesses = []

    for index, row in neighborhood_df.iterrows():
        neighborhood_info = {
            "name": row["name"],
            "location": row["location"],
            "radius": row["radius"]
        }
        neighborhood_business = fetch_business(neighborhood_info, business_type)
        all_businesses.extend(neighborhood_business)

    # Create a DataFrame from the combined list of businesses
    business_df = pd.DataFrame(all_businesses)

    # Save the merged dataset to an Excel file
    business_df.to_excel(excel_filename, index=False)

Testing function on pulling allowed art_gallery data for Baltimore

In [6]:
# To get art_gallery data
fetch_and_save_business_data('art_gallery', 'Allowed_art_gallerys.xlsx')

Using fetch and save function to pull allowed police location data for Baltimore

In [4]:
fetch_and_save_business_data('police', 'police.xlsx')

Formating geometry collumn to show just the location coordinates

In [3]:
# testing a function to extract and format the coordinates from the geometry column

def extract_lat_lng(geometry):
    """
    Extracts latitude and longitude values from the geometry column.
    """
    # Convert string representation of dictionary to a dictionary
    geometry_dict = ast.literal_eval(geometry)
    
    # Extract latitude and longitude values
    lat = geometry_dict['location']['lat']
    lng = geometry_dict['location']['lng']
    
    # Return as a string
    return f"{lat},{lng}"


Filtering out location data for Restaurants

In [4]:
# Read the XLSX file into a DataFrame
rest = pd.read_excel('Allowed_restaurant.xlsx')

# Apply the extraction function to the 'geometry' column
rest['geometry'] = rest['geometry'].apply(extract_lat_lng)

# Save the DataFrame as a CSV file
rest.to_csv('Allowed_restaurantr.csv', index=False)

Filtering out location data for Art Gallery

In [5]:
# Reading the XLSX file into a DataFrame
art = pd.read_excel('Allowed_art_gallerys.xlsx')

# Applying the extraction function to the 'geometry' column
art['geometry'] = art['geometry'].apply(extract_lat_lng)

# Saving the DataFrame as a CSV file
art.to_csv('Filtered_art_gallerys.csv', index=False)

Filtering out location data for Banks

In [6]:
# Reading the XLSX file into a DataFrame
bank = pd.read_excel('Allowed_banks.xlsx')

# Applying the extraction function to the 'geometry' column
bank['geometry'] = bank['geometry'].apply(extract_lat_lng)

# Saving the DataFrame as a CSV file
bank.to_csv('Filtered_banks.csv', index=False)